**This notebook includes the following sections:**

**#01** Importing libraries and data

**#02** Wrangling data

**#03** Merging data and additional wrangling

**#04** Data cleaning and export

**#01 Importing libraries and data**

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
#Importing listing data
path = '/Users/gingermoore/Documents/06-2025 Airbnb Dallas/02 Data/Transformed Data'

In [5]:
df_list = pd.read_pickle(os.path.join(path, 'listings_clean.pkl'))

In [6]:
df_list.shape

(5349, 41)

In [7]:
df_list.head()

id                                               name  \
0    61878                      MODERN LIVING AND FURNISHINGS   
1   795703           Amazing location walk to Downtown Dallas   
2   826118                      Far North Dallas -- Blue Room   
3   860248       Peaceful Home By the Lake: Safe, Cozy, Quiet   
4  1154424  Uptown, Charming Studio 1B,Fast Free WiFi Parking   

                                         description  \
0  Close to downtown and Uptown.  Fast and conven...   
1  ***** Over 30% Discounts for stays over 30 day...   
2                                                nan   
3  Traveling nurses, SMU students welcome - 10% d...   
4  Welcome to the Knox District, a neighborhood t...   

                               neighborhood_overview  host_id host_since  \
0  Enjoy the heart of Dallas right out your door ...   300211 2010-11-26   
1  Located in Turtle Creek neighborhood steps fro...  4191322 2012-11-19   
2                                                nan   804559 2011-07-11   
3  Wonderful big trees and nice sidewalks through...  4505460 2012-12-27   
4  Great Community to Live, Work and Play<br />Lo...  6063232 2013-04-23   

  host_location  host_response_time  host_response_rate  host_acceptance_rate  \
0   Roanoke, TX  within a few hours               100.0                  88.0   
1   Memphis, TN  within a few hours               100.0                 100.0   
2    Dallas, TX        within a day                50.0                   0.0   
3    Dallas, TX      within an hour               100.0                  97.0   
4    Dallas, TX  within a few hours               100.0                  94.0   

   ... review_scores_communication  review_scores_location  \
0  ...                        4.93                    4.75   
1  ...                        4.97                    4.91   
2  ...                        4.85                    4.81   
3  ...                        4.93                    4.92   
4  ...                        4.63                    4.94   

   review_scores_value calculated_host_listings_count  \
0                 4.78                              1   
1                 4.76                              1   
2                 4.78                              1   
3                 4.83                              2   
4                 4.66                              9   

   calculated_host_listings_count_entire_homes  \
0                                            1   
1                                            1   
2                                            0   
3                                            0   
4                                            9   

   calculated_host_listings_count_private_rooms  \
0                                             0   
1                                             0   
2                                             1   
3                                             2   
4                                             0   

  calculated_host_listings_count_shared_rooms reviews_per_month  \
0                                           0              0.32   
1                                           0              0.48   
2                                           0              0.23   
3                                           0              0.68   
4                                           0              0.67   

   host_is_superhost_bool  instant_book_bool  
0                    True              False  
1                   False              False  
2                   False              False  
3                    True              False  
4                   False               True  

[5 rows x 41 columns]

In [8]:
df_list.rename(columns = {'id' : 'listing_id'}, inplace = True)

In [9]:
#Checking to make sure there are 5349 unique listings
df_list['listing_id'].nunique()

5349

In [10]:
#Importing calendar data
df_cal = pd.read_pickle (os.path.join(path, 'calendar_wrangled.pkl'))

In [11]:
df_cal.head()

listing_id       date available  price  minimum_nights  maximum_nights  \
0      61878 2025-04-17         f   75.0            30.0          1125.0   
1      61878 2025-04-18         f   75.0            30.0          1125.0   
2      61878 2025-04-19         f   75.0            30.0          1125.0   
3      61878 2025-04-20         f   75.0            30.0          1125.0   
4      61878 2025-04-21         f   75.0            30.0          1125.0   

   available_bool  
0           False  
1           False  
2           False  
3           False  
4           False

In [12]:
df_cal.shape

(1952365, 7)

**#02 Wrangling data**

In [13]:
#Quickly checking the data types
df_cal.dtypes

listing_id                object
date              datetime64[ns]
available                 object
price                    float64
minimum_nights           float64
maximum_nights           float64
available_bool              bool
dtype: object

In [14]:
#Dropping the available column

In [15]:
df_calendar = df_cal.drop(columns = 'available')

In [16]:
#Checking this one to make sure there are the same number of unique listing ids
df_calendar['listing_id'].nunique()

5349

**#03 Merging data and additional wrangling**

In [17]:
df_merge = df_list.merge(df_calendar, on = 'listing_id', indicator = True)

In [18]:
df_merge.columns

Index(['listing_id', 'name', 'description', 'neighborhood_overview', 'host_id',
       'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price_x',
       'minimum_nights_x', 'maximum_nights_x', 'number_of_reviews',
       'estimated_revenue_l365d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'host_is_superhost_bool', 'insta

In [19]:
df_merge['_merge'].value_counts()

_merge
both          1952365
left_only           0
right_only          0
Name: count, dtype: int64

In [20]:
df_merge['listing_id'].nunique()

5349

This confirms that the listing IDs have joined correctly. Let's check and see if the other columns that copied over are consistent.

In [22]:
df_match = df_merge[['price_x', 'minimum_nights_x', 'maximum_nights_x', 'price_y', 'minimum_nights_y', 'maximum_nights_y']]

In [23]:
df_match.head()

price_x  minimum_nights_x  maximum_nights_x  price_y  minimum_nights_y  \
0     80.0                30              1125     75.0              30.0   
1     80.0                30              1125     75.0              30.0   
2     80.0                30              1125     75.0              30.0   
3     80.0                30              1125     75.0              30.0   
4     80.0                30              1125     75.0              30.0   

   maximum_nights_y  
0            1125.0  
1            1125.0  
2            1125.0  
3            1125.0  
4            1125.0

In [24]:
#So far so good...let's check the descriptive stats
df_match.describe()

price_x  minimum_nights_x  maximum_nights_x       price_y  \
count  1.808944e+06      1.952365e+06      1.952365e+06  1.952365e+06   
mean   2.053996e+02      8.286669e+00      4.692928e+02  3.662788e+02   
std    6.275152e+02      1.765272e+01      3.547559e+02  1.168279e+03   
min    9.000000e+00      1.000000e+00      1.000000e+00  1.000000e+01   
25%    8.000000e+01      1.000000e+00      3.650000e+02  9.000000e+01   
50%    1.160000e+02      2.000000e+00      3.650000e+02  1.370000e+02   
75%    1.850000e+02      4.000000e+00      3.650000e+02  2.600000e+02   
max    1.500000e+04      3.650000e+02      1.125000e+03  1.500000e+04   

       minimum_nights_y  maximum_nights_y  
count      1.952365e+06      1.952365e+06  
mean       9.078510e+00      2.409540e+06  
std        1.805789e+01      7.188325e+07  
min        1.000000e+00      1.000000e+00  
25%        1.000000e+00      3.650000e+02  
50%        2.000000e+00      3.650000e+02  
75%        7.000000e+00      1.125000e+03  
max        3.650000e+02      2.147484e+09

This shows that these variables are not consistent across both data sets, something I failed to check before merging them previously. According to the data dictionary for these sets (which I will submit along with this notebook), the min/max nights are different in the calendar than in the listing database. As far as I can tell, the calendar data will show the min/max number of nights for a booking made on a specific day, and the listing data shows the min/max for that listing in general. As for price, that probably varies based on time of year--a booking made around a major holiday might be more expensive than, say, one random Tuesday in February.

All this being said, I think I'll keep the min/max from the listing dataset as I'm more interested in knowing that information in a general sense. Specificity in this variable isn't necessary for my analysis. I'll keep the price variable from the calendar dataset, since I want more specificity with price, especially as it relates to time of year.

Separately, I noticed there are three different columns that appear to show the total number of listings for a particular host. These are 'host_listings_count', 'host_total_listings_count', and 'calculated_host_listings_count'. I reviewed the data dictionary and found that the first two are based on some unknown calculation that Airbnb has made. The other, 'calculated_host_listings_count', is the number of listings the host has based on the most recent scrape, within the geographic area this dataset focuses on. I'm going to keep that variable as the total number of listings per host.

In [27]:
df_merge_2 = df_merge.drop(columns = ['price_x', 'minimum_nights_y', 'maximum_nights_y', 'host_listings_count', 'host_total_listings_count',
                                      '_merge', 'estimated_revenue_l365d'])

In [28]:
df_merge_2.columns

Index(['listing_id', 'name', 'description', 'neighborhood_overview', 'host_id',
       'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'minimum_nights_x', 'maximum_nights_x', 'number_of_reviews',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'host_is_superhost_bool', 'instant_book_bool', 'date', 'price_y',
       'available_bool'],
      dtype='object')

In [29]:
df_merge_2.rename(columns = {
    'minimum_nights_x' : 'min_nights',
    'maximum_nights_x' : 'max_nights',
    'price_y' : 'price',
    'neighbourhood_cleansed' : 'district',
    'review_scores_rating' : 'rating_score',
    'review_scores_accuracy' : 'accuracy_score',
    'review_scores_cleanliness' : 'clean_score',
    'review_scores_checkin' : 'checkin_score',
    'review_scores_communication' : 'comm_score',
    'review_scores_location' : 'location_score',
    'review_scores_value' : 'value_score',
    'calculated_host_listings_count' : 'host_listings',
    'calculated_host_listings_count_entire_homes' : 'host_entire_homes',
    'calculated_host_listings_count_private_rooms' : 'host_private_rooms',
    'calculated_host_listings_count_shared_rooms' : 'host_shared_rooms',
    'host_is_superhost_bool' : 'superhost_bool'
}, inplace = True)

**#04 Data cleaning and export**

In [30]:
#Let's check for missing values.
df_merge_2.isnull().sum()

listing_id                    0
name                          0
description                   0
neighborhood_overview         0
host_id                       0
host_since                  365
host_location                 0
host_response_time            0
host_response_rate       166429
host_acceptance_rate     118260
host_neighbourhood            0
district                      0
latitude                      0
longitude                     0
property_type                 0
room_type                     0
accommodates                  0
bathrooms                143786
bedrooms                  21170
beds                     143786
min_nights                    0
max_nights                    0
number_of_reviews             0
rating_score             280685
accuracy_score           280685
clean_score              280685
checkin_score            280685
comm_score               280685
location_score           280685
value_score              280685
host_listings                 0
host_ent

There are a lot of missing values, so here's how I'll address them. 
* host_since only has one listing that's missing a date, so I'll leave that one alone.
* host_response_rate and host_acceptance_rate have a lot of missing values. I think I'll impute the average rate for those, though I'll do some checks to see if I should use mean/median/mode.
* bathooms, bedrooms, and beds I'm not too sure about. I feel like I can extrapolate the number of beds based on the number of people the listing accommodates, but I don't think I can do the same about bedrooms and bathrooms. I'll check the min/max values for those and see if it makes sense to impute the average number or just ignore the nulls and move on. I think it makes sense that there would be at least one of each in every listing...
* The ratings I do want to impute, but I think I'll check first to make sure that the null values in question don't just mean that the listing has no reviews at all.
* There are a lot of missing values for superhost_bool. I think I can safely ignore them though.

In [32]:
#First checking the descriptive stats on host_response_rate and host_acceptance_rate
df_merge_2[['host_response_rate', 'host_acceptance_rate']].describe()

host_response_rate  host_acceptance_rate
count        1.785936e+06          1.834105e+06
mean         9.553401e+01          9.108030e+01
std          1.666275e+01          1.997768e+01
min          0.000000e+00          0.000000e+00
25%          1.000000e+02          9.300000e+01
50%          1.000000e+02          9.900000e+01
75%          1.000000e+02          1.000000e+02
max          1.000000e+02          1.000000e+02

In [33]:
#Let's check the median
df_merge_2[['host_response_rate', 'host_acceptance_rate']].median()

host_response_rate      100.0
host_acceptance_rate     99.0
dtype: float64

In [34]:
#Now let's check the mode
df_merge_2[['host_response_rate', 'host_acceptance_rate']].mode()

host_response_rate  host_acceptance_rate
0               100.0                 100.0

Based on this, I think there are some outliers dragging down the mean for both columns. I'll impute the median value for both columns.df_merge_2['host_acceptance_rate'].fillna(df_merge_2['host_acceptance_rate'].median(), inplace = True)

In [36]:
df_merge_2['host_response_rate'].fillna(df_merge_2['host_response_rate'].median(), inplace = True)
df_merge_2['host_acceptance_rate'].fillna(df_merge_2['host_acceptance_rate'].median(), inplace = True)

/var/folders/dt/_2cmd91j32bgtvjmx8m320w40000gn/T/ipykernel_45231/326298556.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merge_2['host_response_rate'].fillna(df_merge_2['host_response_rate'].median(), inplace = True)
/var/folders/dt/_2cmd91j32bgtvjmx8m320w40000gn/T/ipykernel_45231/326298556.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the interme

Let's move on to beds, bathrooms and bedrooms. I think it's reasonable to impute the number of people who can be accomodated for the number of beds. As for imputing bedrooms and bathrooms, I'm not sure. Let's check the descriptive stats...

In [38]:
df_merge_2[['bathrooms', 'bedrooms', 'beds']].describe()

bathrooms      bedrooms          beds
count  1.808579e+06  1.931195e+06  1.808579e+06
mean   1.634610e+00  1.892659e+00  2.557819e+00
std    9.351000e-01  1.316993e+00  2.138974e+00
min    0.000000e+00  0.000000e+00  0.000000e+00
25%    1.000000e+00  1.000000e+00  1.000000e+00
50%    1.000000e+00  1.000000e+00  2.000000e+00
75%    2.000000e+00  3.000000e+00  3.000000e+00
max    1.250000e+01  3.300000e+01  3.900000e+01

According to these stats, there are listings that have zero. In this case, I think I'll leave these nulls alone.

In [40]:
#Let's check the minimum number of reviews a listing has--we know there are no null values.
df_merge_2['number_of_reviews'].describe()

count    1.952365e+06
mean     4.261348e+01
std      8.260969e+01
min      0.000000e+00
25%      2.000000e+00
50%      1.400000e+01
75%      4.500000e+01
max      1.312000e+03
Name: number_of_reviews, dtype: float64

Some of our listings have zero reviews! Let's check if this coincides with the null values of the ratings.

In [42]:
#Creating a subset where only the listings with zero reviews show
review = df_merge_2[df_merge_2['number_of_reviews'] == 0]

In [43]:
review.shape

(280685, 40)

There are the same number of rows in this dataset as there are null values for the rating categories.

In [45]:
review.head()

listing_id                                       name  \
13870    6017390  Discover Dallas: M-Streets Studio Getaway   
13871    6017390  Discover Dallas: M-Streets Studio Getaway   
13872    6017390  Discover Dallas: M-Streets Studio Getaway   
13873    6017390  Discover Dallas: M-Streets Studio Getaway   
13874    6017390  Discover Dallas: M-Streets Studio Getaway   

                                             description  \
13870  Discover urban charm in our 1-bedroom M-Street...   
13871  Discover urban charm in our 1-bedroom M-Street...   
13872  Discover urban charm in our 1-bedroom M-Street...   
13873  Discover urban charm in our 1-bedroom M-Street...   
13874  Discover urban charm in our 1-bedroom M-Street...   

                                   neighborhood_overview   host_id host_since  \
13870  **Explore the M-Streets Neighborhood:**<br /><...  31234076 2015-04-14   
13871  **Explore the M-Streets Neighborhood:**<br /><...  31234076 2015-04-14   
13872  **Explore the M-Streets Neighborhood:**<br /><...  31234076 2015-04-14   
13873  **Explore the M-Streets Neighborhood:**<br /><...  31234076 2015-04-14   
13874  **Explore the M-Streets Neighborhood:**<br /><...  31234076 2015-04-14   

      host_location  host_response_time  host_response_rate  \
13870    Dallas, TX  a few days or more                 0.0   
13871    Dallas, TX  a few days or more                 0.0   
13872    Dallas, TX  a few days or more                 0.0   
13873    Dallas, TX  a few days or more                 0.0   
13874    Dallas, TX  a few days or more                 0.0   

       host_acceptance_rate  ... host_listings host_entire_homes  \
13870                  99.0  ...             1                 1   
13871                  99.0  ...             1                 1   
13872                  99.0  ...             1                 1   
13873                  99.0  ...             1                 1   
13874                  99.0  ...             1                 1   

       host_private_rooms  host_shared_rooms reviews_per_month superhost_bool  \
13870                   0                  0               NaN          False   
13871                   0                  0               NaN          False   
13872                   0                  0               NaN          False   
13873                   0                  0               NaN          False   
13874                   0                  0               NaN          False   

       instant_book_bool       date  price  available_bool  
13870              False 2025-04-17  147.0            True  
13871              False 2025-04-18  147.0            True  
13872              False 2025-04-19  147.0            True  
13873              False 2025-04-20  147.0            True  
13874              False 2025-04-21  147.0            True  

[5 rows x 40 columns]

In [46]:
review.isnull().sum()

listing_id                    0
name                          0
description                   0
neighborhood_overview         0
host_id                       0
host_since                    0
host_location                 0
host_response_time            0
host_response_rate            0
host_acceptance_rate          0
host_neighbourhood            0
district                      0
latitude                      0
longitude                     0
property_type                 0
room_type                     0
accommodates                  0
bathrooms                 24820
bedrooms                   4380
beds                      25185
min_nights                    0
max_nights                    0
number_of_reviews             0
rating_score             280685
accuracy_score           280685
clean_score              280685
checkin_score            280685
comm_score               280685
location_score           280685
value_score              280685
host_listings                 0
host_ent

It looks like the null rating scores correspond to having no reviews. I could impute all the nulls with zeroes, but I think that would mess with calculations later on in my analysis--like if I wanted to get an average rating or something. I think it makes more sense to leave them null.

In [114]:
#Let's check for mixed data types.
for col in df_merge_2.columns.tolist():
    weird = (df_merge_2[[col]].map(type) !=df_merge_2[[col]].
iloc[0].apply(type)).any(axis = 1)
    if len (df_merge_2[weird]) > 0:
        print (col)

host_since
superhost_bool


In [124]:
#Let's find out how many data types we have in host_since.
df_merge_2['host_since'].map(type).unique()

array([<class 'pandas._libs.tslibs.timestamps.Timestamp'>,
       <class 'pandas._libs.tslibs.nattype.NaTType'>], dtype=object)

In [126]:
#And how many we have in superhost_bool
df_merge_2['superhost_bool'].map(type).unique()

array([<class 'bool'>, <class 'float'>], dtype=object)

It looks like in both of these columns, the nulls are counting as mixed data. In that case, we'll leave them alone.

In [137]:
#let's check for full duplicates
df_dups = df_merge_2[df_merge_2.duplicated()]

In [141]:
df_dups.shape

(0, 40)

No duplicates! I think I'll do a check on the price variable and see if there are any extreme values. That's the only variable I'm concerned about having outliers that would affect the analysis.

In [145]:
#Checking descriptive statistics
df_merge_2['price'].describe()

count    1.952365e+06
mean     3.662788e+02
std      1.168279e+03
min      1.000000e+01
25%      9.000000e+01
50%      1.370000e+02
75%      2.600000e+02
max      1.500000e+04
Name: price, dtype: float64

In [149]:
#The std is 1168, but the mean is 366. There must be a lot of outliers. Let's find out how many...
#Three std above the mean is roughly 3871.1158. Let's round up to 3900.
df_outlier = df_merge_2[df_merge_2['price'] >= 3900]
df_outlier.shape

(29565, 40)

In [151]:
#Just as a reminder, here's how many values we have overall...
df_merge_2.shape

(1952365, 40)

That means roughly 1.5% of our dataset could be considered an outlier as far as price. I'm going to go ahead and drop these outliers so we don't have to worry about them affecting our datset negatively.

In [156]:
df_merge_final = df_merge_2[df_merge_2['price'] < 3900]

In [158]:
df_merge_final.shape

(1922800, 40)

In [164]:
df_merge_final.columns

Index(['listing_id', 'name', 'description', 'neighborhood_overview', 'host_id',
       'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_neighbourhood',
       'district', 'latitude', 'longitude', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'min_nights',
       'max_nights', 'number_of_reviews', 'rating_score', 'accuracy_score',
       'clean_score', 'checkin_score', 'comm_score', 'location_score',
       'value_score', 'host_listings', 'host_entire_homes',
       'host_private_rooms', 'host_shared_rooms', 'reviews_per_month',
       'superhost_bool', 'instant_book_bool', 'date', 'price',
       'available_bool'],
      dtype='object')

In [166]:
#Let's export the data now.
df_merge_final.to_pickle(os.path.join(path, 'CleanDallasDataFinal.pkl'))